In [1]:
from generate_elliptic import *
import json
#autoreload
import pandas as pd
%load_ext autoreload
%autoreload 2
import torch.nn as nn

In [3]:
with open("config.json", 'r') as file:
        cfg = json.load(file)
data_df, emb = load_raw_data(cfg)   
data_df = preprocess_data_df(data_df)
print("Generating Train Test")
train_df, test_df, mapping_receivers,mapping_senders = generate_train_test(data_df, cfg)

print("Generating Receivers and Senders List")
tr_receivers, tr_senders_list = get_receivers_to_senders_list(train_df)
tst_receivers, tst_senders_list = get_receivers_to_senders_list(test_df)

print("Creating Train and Test Files")
create_txt_file(tr_receivers, tr_senders_list, f"{GLOBAL_OUTPUT_DATA}{cfg['output_paths']['train_file']}")
create_txt_file(tst_receivers, tst_senders_list, f"{GLOBAL_OUTPUT_DATA}{cfg['output_paths']['test_file']}")


Loading raw data...
Loading node_idx_map...
Loading emb...
Generating Train Test
Number of Receivers:  207
Number of Senders:  226
Generating Receivers and Senders List
Number of all senders: 210
Number of all receivers: 195
Number of all senders: 59
Number of all receivers: 56
Creating Train and Test Files


/skunk-pod-storage-mohamed-2eali-2edhraief-40ibm-2ecom-pvc/NGCF-PyTorch/data/generate_elliptic.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_1_1['senders_mapped'] = data_df_1_1['senders_mapped'].apply(lambda x: list(x)[0])
/skunk-pod-storage-mohamed-2eali-2edhraief-40ibm-2ecom-pvc/NGCF-PyTorch/data/generate_elliptic.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_1_1['receivers_mapped'] = data_df_1_1['receivers_mapped'].apply(lambda x: list(x)[0])


In [5]:
print("Generating Embeddings")
mapping_receivers,neg_rec = augment_with_negatives(data_df, mapping_receivers,"receivers")
emb_receiver = generate_emb(emb, mapping_receivers, f"{GLOBAL_OUTPUT_DATA}{cfg['output_paths']['emb_receivers']}")
print("Shape of Emb Receivers", emb_receiver.shape)


Generating Embeddings
Original Length of Senders: 207
Added receivers: 36588
New Length of Senders: 36633
36632 36632
Shape of Emb Receivers torch.Size([36633, 43])


In [8]:
rec = data_df[data_df.labels==0].receivers_mapped.explode().unique()
for r in rec:
    if r not in mapping_receivers.keys():
        print(r)

In [9]:
mapping_senders,neg_senders = augment_with_negatives(data_df, mapping_senders,"senders")
emb_senders = generate_emb(emb, mapping_senders, f"{GLOBAL_OUTPUT_DATA}{cfg['output_paths']['emb_senders']}")
print("Shape of Emb Senders", emb_senders.shape)

Original Length of Senders: 226
Added senders: 52583
New Length of Senders: 52793
52792 52792
Shape of Emb Senders torch.Size([52793, 43])


In [12]:
(len(neg_rec),len(neg_senders))

(36426, 52567)

In [13]:
with open(f"{GLOBAL_OUTPUT_DATA}{cfg['output_paths']['neg_senders']}",'rb') as f:
            neg_receivers,neg_items = pickle.load(f)

In [17]:
len(neg_receivers),len(neg_items)

(36426, 52567)

In [3]:
import torch
from einops import einsum

# random torch of size 2,3
a = torch.rand(2,172)
b = torch.rand(2,172)

scores = einsum(
            a,b, "i d, j d -> i j"
)
scores

tensor([[45.2998, 47.7303],
        [43.2965, 44.0736]])

In [2]:
torch.load("/skunk-pod-storage-mohamed-2eali-2edhraief-40ibm-2ecom-pvc/NGCF-PyTorch/Data/elliptic/emb_receivers.pt").shape

torch.Size([36633, 43])